<p style='
  color: #3b4045; 
  text-align: center;
  font-weight: bold;
  font-family: -apple-system,BlinkMacSystemFont, "Segoe UI Adjusted","Segoe UI","Liberation Sans",sans-serif;     font-size: 2.07692308rem; '> 
    Vascular Graph Extraction
</p>


In [ ]:
%load_ext autoreload
%autoreload 2

from jppype import Mosaic, imshow, vscode_theme

from fundus_vessels_toolkit.pipelines import FundusVesselSegToGraph
from fundus_vessels_toolkit.segment import segment_vessels
from fundus_vessels_toolkit.utils.data_io import load_image

vscode_theme()

In [2]:
# Path to the raw fundus image
RAW_PATH = "PATH/TO/FUNDUS/IMAGE"

# Path to a vessel ground truth (for comparison purposes)
VESSELS_PATH = "PATH/TO/VESSEL/GROUND/TRUTH"

### Load the raw Image and the Vascular Segmentation


Load image and segmentation mask


In [3]:
raw = load_image(RAW_PATH)
true_vessels = load_image(VESSELS_PATH, binarize=True)

Perform segmentation using a pretrained model


In [4]:
vessels = segment_vessels(raw)

---


### Skeletonize and Extract Graph


All the skeletonization and graph extraction pipeline is accessible through `RetinalVesselSeg2Graph` objects.

This object tunes all the parameters of the pipeline according to a single parameter: the maximum vessel diameter.
Apriori, a good approximation is a 80th of the image width (13px for a 1024x1024 px image).


In [5]:
max_vessel_diameter = raw.shape[1] // 80
seg2graph = FundusVesselSegToGraph(max_vessel_diameter)

To compute the graph extraction you simply need to call the object with a segmentation map:


In [6]:
graph_gt = seg2graph(true_vessels)
graph = seg2graph(vessels)

The returned object is a `VascularGraph` which expose all the properties of the extracted graph.

- The list of branches : `vgrapp.branch_list`
- The y and x position of the nodes: `vgraph.nodes_coord()`

This object is also usefull to cast the graph to other representation. For example:

- Node Adjacency Matrix: `vgraph.node_adjacency_matrix()`
- Branch Adjacency Matrix: `vgraph.branch_adjacency_matrix()`


#### Display


Lets compare the graph extracted on the predicted segmentation and on the ground truth segmentation side by side!

_(The prediction is on the left, the ground truth on the right.)_


In [ ]:
def setup_view(view, vessels_map, vgraph):
    view.add_label(vessels_map, "vessel", "white", opacity=0.3)
    view.add_label(seg2graph.skeletonize(vessels_map), "skel", "grey", opacity=0.3)
    view["vessel graph"] = vgraph.jppype_layer(edge_map=True, edge_labels=False)


mosaic = Mosaic(2, background=raw, cell_height=650, sync=True, cols_titles=["IA Segmentation", "Ground Truth"])
setup_view(mosaic[0], vessels, graph)
setup_view(mosaic[1], true_vessels, graph_gt)

mosaic.show()

---


### Performing only the skeletonization


The method `seg2graph()` actually perform the graph extraction in three steps:

- Skeletonization (morphological reduction of the segmentation to the vessels centerline): `seg2graph.skeletonize(vessel_segmentation_map)`
- Graph extraction (identification of branches and nodes in the vascular skeleton and connection parsing): `seg2graph.skel_to_vgraph(vessel_skeleton_map, vessel_segmentation_map)`.
- Graph simplification (removal of small branches, merging close nodes...): `seg2graph.simplify_vgraph(vgraph)`

Those method can be called individually, for example to get the result of the skeletonization step only.


In [8]:
skeleton = seg2graph.skeletonize(vessels)

You may notice some difference between the skeleton obtained through `seg2graph.skeletonize(vessels)` and by direct call of `seg2graph(vessels)`, because several topological simplification is performed on the latter (merging close nodes, removing small spurs or cycles...).


In [ ]:
v = imshow(raw)
v.add_label(vessels, "vessel", "white", options={"opacity": 0.2})
v.add_label(skeleton, "skeleton", {1: "white", 2: "yellow"})

v.show()